### Impoting the libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

### Loading the dataset 

In [10]:
class DrugDataset(Dataset):
    def __init__(self, data_path, train=True, test_size=0.3):
        df = pd.read_csv(data_path)  # Load dataset
        X = df.iloc[:, :-1].values  # Features (all columns except last)
        y = df.iloc[:, -1].values  # Labels (last column)

        # Split into 70% train and 30% test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        if train:
            self.data, self.labels = X_train, y_train
        else:
            self.data, self.labels = X_test, y_test
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)


### Custom Transformer Model 

In [11]:
class CustomSelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super(CustomSelfAttention, self).__init__()
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.scale = embed_dim ** 0.5
    
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attn_weights = torch.softmax((Q @ K.transpose(-2, -1)) / self.scale, dim=-1)
        return attn_weights @ V

class CustomTransformerBlock(nn.Module):
    def __init__(self, embed_dim):
        super(CustomTransformerBlock, self).__init__()
        self.attention = CustomSelfAttention(embed_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.norm2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        attn_out = self.attention(x)
        x = self.norm1(x + attn_out)
        ff_out = self.ff(x)
        return self.norm2(x + ff_out)



### Model Training 

In [12]:
class DrugClassifier(nn.Module):
    def __init__(self, input_dim, embed_dim, num_classes, num_layers=3):
        super(DrugClassifier, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)  # Convert input features to embeddings
        self.transformer_blocks = nn.ModuleList([CustomTransformerBlock(embed_dim) for _ in range(num_layers)])
        self.fc = nn.Linear(embed_dim, num_classes)
    
    def forward(self, input_ids):
        x = self.embedding(input_ids)
        for block in self.transformer_blocks:
            x = block(x)
        pooled_output = x.mean(dim=1)  # Mean pooling
        return self.fc(pooled_output)


### Testing and Evaluating

In [13]:
def train_and_evaluate(data_path, num_epochs=10, batch_size=8, learning_rate=0.001):
    # Load training and testing datasets (70-30 Split)
    train_dataset = DrugDataset(data_path, train=True)
    test_dataset = DrugDataset(data_path, train=False)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model Initialization
    input_dim = train_dataset.data.shape[1]  # Number of features
    embed_dim = 128  # Embedding size
    num_classes = len(set(train_dataset.labels))  # Unique class labels

    model = DrugClassifier(input_dim, embed_dim, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, labels = batch
            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

    # Testing (Evaluate Model Performance)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, labels = batch
            outputs = model(input_ids)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


### Run Training & Testing

In [14]:
if __name__ == "__main__":
    train_and_evaluate("drug_classification.csv")  # Provide the correct dataset file

FileNotFoundError: [Errno 2] No such file or directory: 'drug_classification.csv'